# Setup inicial

In [3]:
import numpy as np
import pandas as pd
from collections import defaultdict

%matplotlib inline
from matplotlib import pyplot as plt

import seaborn as sns

from IPython.display import HTML, display
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn.model_selection import train_test_split

from sklearn_pandas import DataFrameMapper

from src.creditapp.data.raw_data_access import get_application_record

# Loading data

In [29]:
df_clients = get_application_record()

df_clients.drop(['id'], axis=1, inplace=True)

print('data loading was successfully...')

data loading was successfully...


<img src="https://av.sc.com/in/content/images/in-credit-card-banner-1600x490px.jpg" />

# Experiments

**Fecha de creación: 13/05/2020**

**Autores: Santiago Porchietto, Agustín Trulli**

El objetivo del presente cuaderno es la realización de multiples experimentos involucrando modelos de Machine Learning en función de determinar cual es el que mejor funciona para esta problemática en particular.

Como se menciona en el análisis anterior, se busca predecir si una persona incurrirá en incumplimiento de sus deudas contraidas mediante tarjeta de crédito. Para esto creamos la columna target (y).

Todas las features que quedaron en el dataset (menos el id) posterior al preprocessing serán utilizadas como entradas (X).

<u>Metricas a utilizar</u>:

* Precission: Para poder determinar que porcentaje de las veces que dijimos positivo acertamos la predicción. Por si sola es insuficiente (Encontramos 2 positivos que realmente lo son (100% de precission), en un dataset que posee 1000 positivos).

* Recall: Para complementar precission, nos va a decir el porcentaje de positivos que encontramos por sobre los reales.

* F1-Score y AuCRoc: Como métricas que combinan las anteriores.

Con nuestro ejemplo en particular, precission es la métrica que más deberíamos mirar, ya que, en función de las políticas que tome el banco contra los deudores, deberíamos estar bastante seguros de que una persona lo es antes de etiquetarla de ello.

<u>Modelos a evaluar</u>:

* Decission Tree 

* Random Forest

* 3

* 4

## Feature Engineering 

PREGUNTAR POR LOS NULLS DE LOS TRABAJOS (CATEGORICAL)

### Income range

In [33]:
# parentesis no incluye el término
# corchete incluye el término

df_clients.loc[::, 'months_employed_range'] = pd.cut(df_clients.months_employed, 
                                                     list(range(0, 205, 6)), include_lowest='True')

df_clients['months_employed_range'] = df_clients['months_employed_range'].astype(str)
df_clients.loc[df_clients.months_employed_range == 'nan', 'months_employed_range'] = np.nan
df_clients.loc[df_clients.months_employed_range == '(-0.001, 6.0]', 'months_employed_range'] = '[0.0, 6.0]'
df_clients.loc[df_clients.months_employed >= 205, 'months_employed_range'] = '(204.0, 524.0]'

### Has Childs 

In [37]:
# Childs
df_clients.loc[::, 'has_childs'] = 1
df_clients.loc[df_clients.cnt_children == 0, 'has_childs'] = 0

In [ ]:
occupation_type, name_housing_type, name_family_status, name_education_type, name_income_type

In [38]:
df_clients.head(3)

,code_gender,flag_own_car,flag_own_realty,cnt_children,amt_income_total,name_income_type,name_education_type,name_family_status,name_housing_type,months_employed,flag_mobil,flag_work_phone,flag_email,cnt_fam_members,age,flag_phone,occupation_type,target,months_employed_range,has_childs
0,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,151.0,1,1,0,2.0,32,0,NaN,0,"(150.0, 156.0]",0
2,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,38.0,1,0,0,2.0,58,0,Security staff,0,"(36.0, 42.0]",0
3,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,102.0,1,0,1,1.0,52,1,Sales staff,0,"(96.0, 102.0]",0


In [41]:
df_clients.name_family_status.value_counts() # juntado / no juntado

Married                 6323
Single / not married    1350
Civil marriage           831
Separated                566
Widow                    385
Name: name_family_status, dtype: int64

In [42]:
df_clients.name_housing_type.value_counts() # casa propia / no propia

House / apartment      8430
With parents            448
Municipal apartment     322
Rented apartment        144
Office apartment         77
Co-op apartment          34
Name: name_housing_type, dtype: int64

## Generating test, train & validation sets 

In [31]:
# Test, train & validation sets
train, not_train = train_test_split(df_clients, test_size=0.3, random_state=1)
validation, test = train_test_split(not_train, test_size=0.5, random_state=1)

print('data generating was successfully...')

data generating was successfully...
